In [95]:
import random
import math
from copy import deepcopy
import numpy as np


# --- 辅助函数 ---

def euclidean_distance(loc1, loc2):
    """计算两点间的欧几里得距离""" #这个要接如api
    return math.sqrt((loc1[0] - loc2[0]) ** 2 + (loc1[1] - loc2[1]) ** 2)

def check_route_feasibility(route, vehicle):
    """
    检查路线的时间窗和载重约束，并返回总行驶距离（作为成本指标）。
    route 为节点列表，要求 route[0] 与 route[-1]均为车辆起点（终点）。
    若不可行，则返回 None。
    """
    time = 0      # 假设车辆从起点出发的初始时间为 0，可以更改为vessel特有的起始时间
    load = 0
    total_distance = 0
    capacity = vehicle['capacity']
    current = route[0] #vessel 起始位置
    for nxt in route[1:]:
        travel = euclidean_distance(current["location"], nxt["location"]) #距离计算，需要调用api 函数，单位是时间
        total_distance += travel #需要乘以该段对应vessel的cost/h,计算总成本 
        arrival = time + travel #时间
        # 如果提前到达，则等待至时间窗开始
        start_service = max(arrival, nxt["tw"][0])
        if start_service > nxt["tw"][1]:
            return None
        load += nxt["demand"] #装载或者卸货
        if load > capacity or load < 0:
            return None
        time = start_service + nxt["service"]#装卸货时间
        current = nxt   
    return total_distance 

def try_all_insertions(route, pickup_node, delivery_node, vehicle_id, vehicles):
    """
    在已有路线中枚举所有可能的位置插入 pickup_node 和 delivery_node
    （pickup 需插入在 delivery 之前），
    返回最优（增量成本最低）的方案：
         (最小增量成本, 新的路线)
    若没有可行方案，则返回 (None, None)。
    """
    best_cost = None
    best_route = None
    # 至少在起点之后插入 pickup，delivery 插入位置必须在 pickup 之后
    vehicle = vehicles[vehicle_id]
    for i in range(1, len(route)):
        for j in range(i + 1, len(route) + 1):
            new_route = route[:i] + [pickup_node] + route[i:j] + [delivery_node] + route[j:]
            new_cost = check_route_feasibility(new_route, vehicle)
            if new_cost is not None:
                # 原路线成本（新路线创建时原成本记作 0）
                old_cost = check_route_feasibility(route, vehicle) or 0
                incr = new_cost - old_cost
                if best_cost is None or incr < best_cost:
                    best_cost = incr
                    best_route = new_route
    return best_cost, best_route

# --- 简化后的调度算法 ---
def greedy_pdvrp_simple(requests, vehicles, rejection_penalty):
    """
    参数：
      requests: 每个请求为字典，包含：
           "id"      : 请求编号
           "pickup"  : 取货节点，字典格式 { "id": 节点编号,
                                               "type": "pickup",
                                               "location": (x, y),
                                               "tw": (开始时间, 结束时间),
                                               "service": 服务时间,
                                               "demand": 正数 }
           "delivery": 送货节点，字典格式 { "id": 节点编号,
                                               "type": "delivery",
                                               "location": (x, y),
                                               "tw": (开始时间, 结束时间),
                                               "service": 服务时间,
                                               "demand": 负数 }
      vehicles: 每辆车辆为字典，包含：
           "id"      : 车辆编号
           "start"   : 车辆起始节点（格式同 depot 节点，建议 service=0, demand=0）
           "capacity": 车辆最大载重
      rejection_penalty: 拒绝请求的成本惩罚（数值）

    算法思路：
      1. 对于每个请求（按输入顺序），尝试两种调度方式：
          a. 在已有车辆路线中查找最优的插入位置（pickup 与 delivery 均插入）。
          b. 从尚未用过的车辆中，新建一条路线，其形式为：
             [车辆起点, pickup, delivery, 车辆起点]。
      2. 选取两种方式中成本最小的方案，如果该成本小于拒绝惩罚，则采用该方案；
         否则直接拒绝该请求。
      3. 最后返回所有车辆的路线和被拒绝请求列表。
    """
    routes = []            # 已调度的车辆路线，每项格式为 {"vehicle": vehicle, "nodes": [...] }
    available_vehicles = vehicles.copy()  # 尚未使用的车辆
    rejected = []

    # 按请求顺序依次调度请求
    for req in requests:
        best_cost = None
        best_option = None  # ("existing", route_index, new_route) 或 ("new", vehicle, new_route)
        
        # 在已有路线中搜索插入方案
        for idx, route in enumerate(routes):
            cost, new_route = try_all_insertions(route["nodes"], req["pickup"], req["delivery"], route['vehicle']['id'], vehicles)
            if cost is not None:
                if best_cost is None or cost < best_cost:
                    best_cost = cost
                    best_option = ("existing", idx, new_route)
        
        # 在尚未使用的车辆中尝试新建路线
        for v in available_vehicles:
            new_route = [v["start"], req["pickup"], req["delivery"]]
            cost_candidate = check_route_feasibility(new_route, v)
            if cost_candidate is not None:
                if best_cost is None or cost_candidate < best_cost:
                    best_cost = cost_candidate
                    best_option = ("new", v, new_route)
        
        # 如果找到的最优方案成本低于拒绝惩罚，则采用该方案；否则拒绝该请求
        if best_option is None or best_cost >= rejection_penalty*req['pickup']['demand']:
            rejected.append(req)
        # if best_cost >= rejection_penalty*req['pickup']['demand']:
        #     rejected.append(req)
        else:
            if best_option[0] == "existing":
                # 更新对应车辆路线
                idx, new_route = best_option[1], best_option[2]
                routes[idx]["nodes"] = new_route
            elif best_option[0] == "new":
                vehicle, new_route = best_option[1], best_option[2]
                routes.append({"vehicle": vehicle, "nodes": new_route})
                available_vehicles.remove(vehicle)
    
    # # --- 成本计算 ---
    # total_route_distance = 0
    # for r in routes:
    #     dist = check_route_feasibility(r["nodes"], r["vehicle"])
    #     if dist is not None:
    #         total_route_distance += dist
    # total_rejection_cost = sum(rejection_penalty * req["pickup"]["demand"] for req in rejected)
    # total_cost = total_route_distance + total_rejection_cost


    # --- 成本计算 ---
    total_route_distance = 0
    for r in routes:
        dist = check_route_feasibility(r["nodes"], r["vehicle"])
        if dist is not None:
            total_route_distance += dist

    total_rejection_cost = rejection_penalty * len(rejected)
    total_cost = total_route_distance + total_rejection_cost
    return routes, rejected, total_cost




def multi_random_greedy_pdvrp(requests, vehicles, rejection_penalty, k=10, seed=None):
    """
    多次运行 greedy_pdvrp_simple，每次打乱请求顺序，产生 k 个解并返回。
    
    返回值:
        solutions: List of (routes, rejected, total_cost)，按 total_cost 升序排列
    """
    if seed is not None:
        random.seed(seed)

    routes = []
    rejected = []
    total_cost = []

    for _ in range(k):
        # 深拷贝 requests 和 vehicles
        req_copy = requests.copy()
        random.shuffle(req_copy)
        veh_copy = [v.copy() for v in vehicles]
        for v in veh_copy:
            v["start"] = v["start"].copy()

        # 调用原始 greedy 算法
        routes1, rejected1, total_cost1 = greedy_pdvrp_simple(req_copy, veh_copy, rejection_penalty)
        #routes1, rejected1, total_cost1 = greedy_pdvrp_fully_greedy(requests, vehicles, rejection_penalty)
        routes. append(routes1)
        rejected.append(rejected1)
        total_cost.append(total_cost1)

    # 按总成本升序排序

    return routes, rejected, total_cost

In [96]:
#数据生成

def generate_unique_integer_coordinates(n_points, x_max=100, y_max=100, seed=None):
    if seed is not None:
        np.random.seed(seed)
    all_coords = [(x, y) for x in range(x_max + 1) for y in range(y_max + 1)]
    selected_coords = np.random.choice(len(all_coords), size=n_points, replace=False)
    return [all_coords[i] for i in selected_coords]

def generate_time_windows_with_travel_check(R, coords, o_r, d_r, EP_max, ED_max, M_3, none_prob):
    EP, LP, ED, LD = [], [], [], []

    for r in range(R):
        i, j = o_r[r], d_r[r]
        dist = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
        min_time = int(np.ceil(dist))

        ep = np.random.randint(0, EP_max) if np.random.rand() > none_prob else 0
        ed_min = ep + min_time
        ed = np.random.randint(ed_min, ED_max) if np.random.rand() > none_prob else 0

        lp_min = ep + min_time
        lp_max = min(lp_min + 200, M_3)
        lp = np.random.randint(lp_min, lp_max) if np.random.rand() > none_prob else M_3

        ld_min = ed + min_time
        if np.random.rand() > none_prob and ld_min < M_3:
            ld_max = min(ld_min + 200, M_3)
            ld = np.random.randint(ld_min, ld_max)
        else:
            ld = M_3

        EP.append(ep)
        LP.append(lp)
        ED.append(ed)
        LD.append(ld)

    return np.array(EP), np.array(LP), np.array(ED), np.array(LD)


def generate_vehicle_and_requests_from_ports(V=3, R=10, N=10, x_max=100, y_max=100,
                                             EP_max=200, ED_max=500, M_3=1000, none_prob=0.5, seed=None):
    """
    - 生成 N 个港口点（用于 depot + pickup + delivery）
    - 从中抽取 V 个作为车辆起点
    - 从中随机抽取 R 个请求 (o ≠ d)
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # 1. 生成 N 个港口坐标
    coords = generate_unique_integer_coordinates(N, x_max, y_max, seed=seed)

    # 2. 随机抽 V 个港口作为 depot 起点
    depot_indices = random.sample(range(N), V)

    vehicles = []
    for i, idx in enumerate(depot_indices):
        location = coords[idx]
        vehicles.append({
            "id": i,
            "start": {
                "id": f"veh{i+1}_start",
                "type": "depot",
                "location": location,
                "tw": (0, M_3),
                "service": 0,
                "demand": 0
            },
            "capacity": random.randint(10, 20)
        })

    # 3. 为请求抽样起点终点对
    o_r = []
    d_r = []
    for _ in range(R):
        while True:
            o, d = random.sample(range(N), 2)
            if o != d:
                o_r.append(o)
                d_r.append(d)
                break

    # 4. 生成时间窗
    EP, LP, ED, LD = generate_time_windows_with_travel_check(R, coords, o_r, d_r, EP_max, ED_max, M_3, none_prob)

    # 5. 构造请求
    requests = []
    for r in range(R):
        demand = random.randint(3, 8)
        requests.append({
            "id": r + 1,
            "pickup": {
                "id": f"{r+1}p",
                "type": "pickup",
                "location": coords[o_r[r]],
                "tw": (int(EP[r]), int(LP[r])),
                "service": 10,
                "demand": demand
            },
            "delivery": {
                "id": f"{r+1}d",
                "type": "delivery",
                "location": coords[d_r[r]],
                "tw": (int(ED[r]), int(LD[r])),
                "service": 10,
                "demand": -demand
            }
        })

    return vehicles, requests, coords



In [ ]:
# Written by Harry
def calculate_efficiency(route, vehicle, requests, order_idx, request_loading_cost = 0):
    """
    Effciency({RP_v}) = \frac{\sum_{(i,j) \in \text{leg}_v} \text{Cost}_{ijv}}{ \sum_{r \in R_v} c^{\text{load}}_{i_r j_r v}  + c^{\text{load}}_{i_r v r} + c^{\text{unload}}_{j_r v r}}
    """
    current = route[0] #vessel 起始位置
    total_distance = check_route_feasibility(route, vehicle)
    
    absolute_distance = 0
    for order in order_idx:
        order = order-1
        absolute_distance += euclidean_distance(requests[order]["pickup"]["location"], requests[order]["delivery"]["location"])
        # print(order, euclidean_distance(requests[order]["pickup"]["location"], requests[order]["delivery"]["location"]))
    
    efficiency = (total_distance + 2*np.sum(request_loading_cost)) / (absolute_distance + 2*np.sum(request_loading_cost))
    return efficiency

# def trace_orders(route):
#     empty_miles = euclidean_distance(route[0]["location"], route[1]["location"])
#     order_count = 0
#     onboard_orders = []
#     order_cost_dict = {}
#     current = route[1]
#     for node in route[1:]:
#         for order in onboard_orders:
#             order_cost_dict[order]["cost_detail"].append(euclidean_distance(current["location"], node["location"])/len(onboard_orders))
        
#         if len(onboard_orders) == 0:
#             empty_miles += euclidean_distance(current["location"], node["location"])

#         n_id = node["id"]
#         if 'p' in n_id:
#             order_cost_dict[n_id[:-1]] = {}
#             order_cost_dict[n_id[:-1]]["cost_detail"] = []
#             onboard_orders.append(n_id[:-1])
#             order_count += 1
#         if 'd' in n_id:
#             onboard_orders.remove(n_id[:-1])

#         current = node

#     for key in order_cost_dict:
#         order_cost_dict[key]["cost"] = sum(order_cost_dict[key]["cost_detail"]) + empty_miles/order_count
#     print(order_cost_dict)

def trace_orders(route):
    # 初始化当前节点和当前地点
    current = route[0]
    current_location = current["location"]

    # 用于分段存储不同地点的订单ID
    location_count = 0
    locations_dict = {} 
    locations_dict[location_count] = [current["id"]]

    # 遍历route中的每一个节点，按location进行分组
    for node in route[1:]:
        if node["location"] == current_location:
            locations_dict[location_count].append(node["id"])
        else:
            location_count += 1
            locations_dict[location_count] = [node["id"]]

        current_location = node["location"]
    
    # 初始化空驶里程（第一段距离）
    empty_miles = euclidean_distance(route[0]["location"], route[1]["location"])
    order_count = 0; onboard_count = 0
    onboard_orders = []
    order_cost_dict = {}

    # 遍历route，处理每个节点（从第二个开始）
    for node in route[1:]:
        n_id = node["id"]
        distance = euclidean_distance(current["location"], node["location"])
        for order in onboard_orders:
            order_cost_dict[order]["cost_detail"].append(distance)

            # I need to check the one node after is a new pickup node or not 
            pickup_orders = [sum(1 for i in locations_dict[key] if 'p' in i)
                              for key in locations_dict if n_id in locations_dict[key]]

            # 更新当前在车订单数（加上即将上车的pickup orders)
            order_cost_dict[order]["onboard_count"].append(len(onboard_orders)+pickup_orders[0])
        
        # 更新空驶距离和在车订单数量（非0距离代表有移动）
        if distance != 0:
            empty_miles = distance
            onboard_count = len(onboard_orders)

        if 'p' in n_id:
            order_cost_dict[n_id[:-1]] = {}
            order_cost_dict[n_id[:-1]]["cost_detail"] = [max(distance, empty_miles)] # 去pickup的成本
            order_cost_dict[n_id[:-1]]["onboard_count"] = [
                sum(1 for i in locations_dict[key] if 'p' in i)+onboard_count for key in locations_dict if n_id in locations_dict[key]]
            onboard_orders.append(n_id[:-1])
            order_count += 1
        if 'd' in n_id:
            onboard_orders.remove(n_id[:-1])

        current = node
    
    cost_in_total = 0 
    for key in order_cost_dict:
        order_cost_dict[key]["cost"] = 0
        for idx in range(len(order_cost_dict[key]["cost_detail"])):
            # 每段距离成本除以该段时车上订单数
            order_cost_dict[key]["cost"] += order_cost_dict[key]["cost_detail"][idx] / order_cost_dict[key]["onboard_count"][idx]
        cost_in_total += order_cost_dict[key]["cost"]
        
    print(order_cost_dict)
    print(locations_dict)
    return order_cost_dict, cost_in_total
            
def extract_order_id(nodes):
    # —— 下面这一块替换原来的 order_ids 提取 —— #
    seen_orders = set()
    order_indices = []
    for n in nodes:
        if n["type"] == "depot":
            continue
        # n["id"] 形如 "29p" 或 "29d"，我们去掉最后一位后缀
        oid = n["id"][:-1]
        if oid not in seen_orders:
            seen_orders.add(oid)
            order_indices.append(int(oid))  # 转成整数，如果你想保留字符串就用 oid
    return order_indices


In [ ]:
def main():
    seed = 2
    shuffle_time = 50
    Vessel_num = 2
    Request_num = 50
    k_best = 5  # 想展示前几个独特解

    vehicles, requests, port_coords = generate_vehicle_and_requests_from_ports(
        V=Vessel_num, R=Request_num, N=10, seed=seed)

    rejection_penalty = 2000

    # 1. 生成 k 个解
    routes_all, rejected_all, total_cost_all = multi_random_greedy_pdvrp(
        requests, vehicles,
        rejection_penalty=rejection_penalty,
        k=shuffle_time, seed=seed)
    # 2. 去重 + 排序
    seen = set()
    unique_solutions = []
    for routes, rej, cost in zip(routes_all, rejected_all, total_cost_all):
        for trip in routes:
            vehicle = trip["vehicle"]
            nodes   = trip["nodes"]
            order_indices = extract_order_id(nodes)
            efficiency = calculate_efficiency(nodes, vehicle, requests, order_indices)
        if cost not in seen:
            seen.add(cost)
            unique_solutions.append((routes, rej, cost, efficiency))

    unique_solutions.sort(key=lambda x: x[-1])  # 按成本/效率升序

    # 3. 构建 orders_per_solution 和 costs_per_solution
    orders_per_solution = []
    total_costs_per_solution  = []
    detail_costs_per_solution = []

    for routes, rej, cost, efficiency in unique_solutions[:k_best]:
        sol_orders = []
        sol_total_costs  = []
        sol_detail_costs = []
        for trip in routes:
            vehicle = trip["vehicle"]
            nodes   = trip["nodes"]

            order_indices = extract_order_id(nodes)
            total_cost = check_route_feasibility(nodes, vehicle)
            sol_orders.append(order_indices)
            # —— 提取完毕 —— #

            # 每条船的成本
            sol_total_costs.append(check_route_feasibility(nodes, vehicle) or 0.0)
            # efficiency = calculate_efficiency(nodes, vehicle, requests, order_indices)
            order_cost_dict, cost_in_total = trace_orders(nodes)
            sol_detail_costs.append(order_cost_dict)
            assert abs(cost_in_total - total_cost) <= 0.00001, f"cost ({cost_in_total}, {total_cost}) not match!! "
            absolute_cost = 0
            for order in order_indices:
                abs_cost = euclidean_distance(requests[order-1]["pickup"]["location"], requests[order-1]["delivery"]["location"])
                absolute_cost += abs_cost
                print(f'Order: {order}, Absolute cost: {abs_cost}, Marginal cost:{order_cost_dict[str(order)]["cost"]}')

            print(f"Absolute Cost: {absolute_cost}, Cost: {total_cost}, Effciency: {efficiency}")
            
        orders_per_solution.append(sol_orders)
        total_costs_per_solution.append(sol_total_costs)
        detail_costs_per_solution.append(sol_detail_costs)
    
    # Based on the ocurrance number, record all marginal costs
    costs_dict = {}
    for s_idx, sol in enumerate(orders_per_solution):
        for v_idx, vessel in enumerate(sol):
            for order in vessel:
                if order not in costs_dict:
                    costs_dict[order] = {}
                    costs_dict[order]["marginal_costs"] =[]
                    costs_dict[order]["absolute_cost"] = euclidean_distance(requests[order-1]["pickup"]["location"], requests[order-1]["delivery"]["location"])
                costs_dict[order]["marginal_costs"].append(detail_costs_per_solution[s_idx][v_idx][str(order)]["cost"])

    # Bid for an order can be 1/2*(abs_cost + np.mean(marginal_costs)) or np.mean(marginal_costs.extend(abs_cost))
    for o_id in costs_dict:
        costs_dict[o_id]["final_bid"] = 1/2 * (np.mean(costs_dict[o_id]["marginal_costs"]) + costs_dict[o_id]["absolute_cost"])
        print(f'Order: {o_id}, Absolute cost: {costs_dict[o_id]["absolute_cost"]}, Final bid: {costs_dict[o_id]["final_bid"]}')

    # 4. 打印结果（保持不变）
    print("==== 各方案车辆路线（按总成本从低到高） ====")
    for idx, (routes, rej, cost, efficiency) in enumerate(unique_solutions[:k_best]):
        driving_cost = cost - len(rej) * rejection_penalty
        print(f"\n--- 第 {idx+1} 个解 ---")
        print(f"总成本: {cost:.2f}（行驶成本: {driving_cost:.2f}，拒单数: {len(rej)}）")
        for trip in routes:
            vid  = trip['vehicle']['id']
            path = ' → '.join(node['id'] for node in trip['nodes'])
            print(f"车辆 {vid+1} 路径：{path}")

    # （可选）查看处理后的 lists
    print("\norders_per_solution:", orders_per_solution)
    print("total_costs_per_solution:", total_costs_per_solution)

if __name__ == "__main__":
    main()


{'15': {'cost_detail': [43.278170016764804, 60.63827174318213, 36.359317925395686], 'onboard_count': [1, 2, 2], 'cost': 91.7769648510537}, '38': {'cost_detail': [60.63827174318213, 36.359317925395686, 52.32590180780452, 12.041594578792296, 73.16419889536138], 'onboard_count': [2, 2, 2, 3, 4], 'cost': 96.96666032162895}, '43': {'cost_detail': [52.32590180780452, 12.041594578792296, 73.16419889536138, 0.0, 56.0357029044876], 'onboard_count': [2, 3, 4, 3, 3], 'cost': 67.14643312216924}, '16': {'cost_detail': [12.041594578792296, 73.16419889536138, 0.0, 56.0357029044876, 52.32590180780452, 0.0], 'onboard_count': [3, 4, 3, 3, 3, 4], 'cost': 58.42544948753515}, '32': {'cost_detail': [73.16419889536138, 56.0357029044876, 52.32590180780452, 0.0, 44.94441010848846], 'onboard_count': [4, 3, 3, 4, 2], 'cost': 76.88378968218194}, '30': {'cost_detail': [52.32590180780452, 0.0, 44.94441010848846, 31.78049716414141, 34.713109915419565], 'onboard_count': [3, 4, 2, 2, 2], 'cost': 73.16097586329289}, '4